In [1]:
using Revise
using KLIEPInference
using Distributions
using LinearAlgebra

┌ Info: Recompiling stale cache file /home/mkolar/.julia/compiled/v1.0/KLIEPInference/rX42g.ji for KLIEPInference [2424ee94-ddf3-11e8-1bca-091389175f78]
└ @ Base loading.jl:1187


In [2]:
θx = chain(40, 0.1, 0.2)
mask = tril(ones(Bool, size(θx)), -1)
θxv = θx[ mask ]

780-element Array{Float64,1}:
 -0.16043259378527097
  0.0                
  0.0                
  0.0                
  0.0                
  0.0                
  0.0                
  0.0                
  0.0                
  0.0                
  0.0                
  0.0                
  0.0                
  ⋮                  
  0.0                
  0.0                
  0.13418325681820967
  0.0                
  0.0                
  0.0                
  0.11154755407499446
  0.0                
  0.0                
 -0.11534661282060431
  0.0                
  0.14267631626967223

In [3]:
Δv = zeros(length(θxv))
Δv[1] += 0.2 
Δv[5] += 0.3
θyv = θxv - Δv
θy = zeros(Float64, size(θx))
θy[ mask ] = θyv
θy = θy + θy'

40×40 Array{Float64,2}:
  0.0       -0.360433   0.0       …   0.0        0.0       0.0     
 -0.360433   0.0       -0.176943      0.0        0.0       0.0     
  0.0       -0.176943   0.0           0.0        0.0       0.0     
  0.0        0.0       -0.105176      0.0        0.0       0.0     
  0.0        0.0        0.0           0.0        0.0       0.0     
 -0.3        0.0        0.0       …   0.0        0.0       0.0     
  0.0        0.0        0.0           0.0        0.0       0.0     
  0.0        0.0        0.0           0.0        0.0       0.0     
  0.0        0.0        0.0           0.0        0.0       0.0     
  0.0        0.0        0.0           0.0        0.0       0.0     
  0.0        0.0        0.0       …   0.0        0.0       0.0     
  0.0        0.0        0.0           0.0        0.0       0.0     
  0.0        0.0        0.0           0.0        0.0       0.0     
  ⋮                               ⋱                                
  0.0        0.0        

In [4]:
spl = IsingSampler(θx)
X = rand(spl, 400)
spl = IsingSampler(θy)
Y = rand(spl, 600)
Ψx = Ψising(X)
Ψy = Ψising(Y);

In [5]:
p = 10
nx = 400
@show σ = maximum( sqrt.( var(Ψx; dims=2) + var(Ψy; dims=2) ) )
θhat = spKLIEP(Ψx, Ψy, σ * sqrt(2 * log(p) / nx), CD_KLIEP())

σ = maximum(sqrt.(var(Ψx; dims=2) + var(Ψy; dims=2))) = 1.4156891306872295


780-element ProximalBase.SparseIterate{Float64,1} with 16 stored entries:
  [5  ]  =  0.273813
  [8  ]  =  0.00571543
  [35 ]  =  0.000890841
  [151]  =  -0.0189853
  [160]  =  -0.0183499
  [173]  =  -0.0110694
  [273]  =  0.0112009
  [293]  =  0.000156259
  [298]  =  -0.0014962
  [339]  =  -0.0332333
  [343]  =  0.0116729
  [351]  =  0.000216736
  [407]  =  -0.00288175
  [515]  =  0.023624
  [572]  =  -0.00553047
  [641]  =  0.0836175


In [6]:
H = KLIEP_Hessian(θhat, Ψy)

780×780 Array{Float64,2}:
  0.882841     -0.149164     0.0273379   …   0.034353    -0.0305747 
 -0.149164      0.999995    -0.0985989      -0.0704575    0.0200607 
  0.0273379    -0.0985989    0.999255        0.00852835  -0.0319064 
 -0.0203371     0.0148179    0.134916        0.136974    -0.0234429 
 -0.0174418     0.0565867   -0.020089       -0.0584385    0.0196972 
  0.0408054    -0.0321053    0.0616094   …   0.116338    -0.038548  
 -0.0031866    -0.00632837   0.0329315       0.0231229    0.0124233 
 -0.0289116     0.0262724    0.0019339      -0.00897112  -0.0458585 
 -0.031605     -0.0566785    0.0270525       0.0493909    0.0363939 
  0.0443715    -0.00860119   0.0129895       0.106097    -0.00278461
 -0.0292334     0.011352     0.0540542   …  -0.0115009   -0.0567971 
 -0.0466488     0.0649883   -0.0510272      -0.0225436    0.0552985 
 -0.0263437    -0.0199452   -0.0246075       0.149931    -0.0375999 
  ⋮                                      ⋱                          
  0.0330

In [20]:
ny = 600
Hinv_row(H, 6, 1.2 * sqrt(2 * log(p) / ny))

iter = 1
iter = 2
iter = 3
iter = 4


780-element ProximalBase.SparseIterate{Float64,1} with 16 stored entries:
  [6  ]  =  1.01004
  [7  ]  =  -0.0528946
  [44 ]  =  0.243872
  [123]  =  -0.0120789
  [236]  =  -0.0110077
  [269]  =  0.0155054
  [275]  =  0.00559805
  [376]  =  0.000508135
  [409]  =  0.0123722
  [460]  =  0.00755207
  [470]  =  0.00514727
  [494]  =  -0.0112269
  [495]  =  0.0239441
  [694]  =  0.0103084
  [715]  =  0.000564379
  [779]  =  -0.0107582
